# Algo-trading: data preparation


> Author: <b>Nicola Bini</b>
>
> Date:   <b>06/17/2021</b>
>
>
> <b>Team 2:</b>
> <i>
> <br>Nicola Bini
> <br>Felipe Domingues
> <br>Tri Dung Dinh
> <br>Manuel Echazarra
> </i>

# Summary

This is a Keras LSTM model to predict whether or not the stock will close at higher price compared to the closing
price of the previous day.
Its predictions are then saved in a csv file as a signal for the bt library.

In [1]:
### Import libraries ###
from keras.models import Sequential       # Sequential model
from keras.layers import Dense            # Dense layer
from keras.layers import LSTM             # LSTM layer
from keras.layers import Dropout          # Dropout layer
#from tensorflow   import set_random_seed  # set random seed

import pandas            as pd
import numpy             as np
import matplotlib.pyplot as plt

# Set random seed for reproducible results
np.random.seed(42)

In [2]:
# Load data
data = pd.read_csv("./final_data.csv")
data = data.rename(columns = {"Unnamed: 0":"date"})
data = data.set_index("date")
data

,DNN,HGEN,ICLN,OCGN,VGT,VUZI,XLV,^NBI,price%_DNN,price%_HGEN,...,sma_XLV,sma_^NBI,profit_DNN,profit_HGEN,profit_ICLN,profit_OCGN,profit_VGT,profit_VUZI,profit_XLV,profit_^NBI
date,,,,,,,,,,,,,,,,,,,,,
2015-09-18,0.44,15.600000,9.390000,357.000000,102.790001,5.150000,70.910004,3732.669922,0.000000,-0.031056,...,1.014546,1.053607,1.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0
2015-09-21,0.42,14.300000,9.410000,342.600006,103.639999,4.880000,69.970001,3568.250000,-0.045455,-0.083333,...,1.013110,1.051290,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
2015-09-22,0.42,13.200000,9.200000,315.000000,102.019997,4.780000,69.540001,3508.300049,0.000000,-0.076923,...,1.011404,1.048319,1.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0
2015-09-23,0.41,13.700000,9.060000,298.200012,101.180000,4.780000,69.470001,3488.520020,-0.023810,0.037879,...,1.009513,1.044806,0.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0
2015-09-24,0.41,12.150000,9.020000,282.000000,101.080002,5.230000,68.760002,3418.729980,0.000000,-0.113139,...,1.007465,1.040933,1.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-06-02,1.26,19.360001,22.500000,9.455000,372.720001,19.990000,121.330002,4749.950195,-0.045455,0.039184,...,1.067943,1.019637,0.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0
2021-06-03,1.28,18.240000,22.090000,8.890000,368.730011,18.870001,121.699997,4744.620117,0.015873,-0.057851,...,1.068541,1.019038,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
2021-06-04,1.37,18.580000,22.299999,8.720000,375.700012,18.730000,122.089996,4803.450195,0.070313,0.018640,...,1.069211,1.019199,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [3]:
out = data['profit_DNN']

In [4]:
out.head(15)

date
2015-09-18    1.0
2015-09-21    0.0
2015-09-22    1.0
2015-09-23    0.0
2015-09-24    1.0
2015-09-25    1.0
2015-09-28    0.0
2015-09-29    0.0
2015-09-30    0.0
2015-10-01    0.0
2015-10-02    0.0
2015-10-05    0.0
2015-10-06    0.0
2015-10-07    0.0
2015-10-08    0.0
Name: profit_DNN, dtype: float64

In [5]:
# DNN and ICLN Energy
# OCGN and ^NBI Biotechonlogy
# HGEN and XLV Healthcare
# VUGI and VGT Vision 

ticket     = 'DNN'
index_fund = 'ICLN'

lookback = 15 # Days
split_date = '2018-06-01' # Data goes from '2015-09-18' to '2021-06-08'

drop_columns = ["profit_" + index_fund,ticket,index_fund]
for col in data.columns:
    #print(col)
     if ticket not in col and index_fund not in col:
            drop_columns.append(col)

        
df_data = data.drop(drop_columns, axis=1)

df_train = df_data[df_data.index < split_date]
df_test  = df_data[df_data.index > split_date]

train_date = df_train.reset_index()
train_date = train_date['date']
train_date
test_date = df_test.reset_index()
test_date = test_date['date']
test_date

0      2018-06-04
1      2018-06-05
2      2018-06-06
3      2018-06-07
4      2018-06-08
          ...    
754    2021-06-02
755    2021-06-03
756    2021-06-04
757    2021-06-07
758    2021-06-08
Name: date, Length: 759, dtype: object

In [6]:
x_train.to_excel("presentation.xlsx")

NameError: name 'x_train' is not defined

In [ ]:
x_train = df_train.drop(["profit_" + ticket], axis=1)
y_train = df_train['profit_' + ticket]

x_test = df_test.drop(["profit_" + ticket], axis=1)
y_test = df_test['profit_' + ticket]

In [ ]:
def create_dataset_with_lookback(data, lookback):
    
    data = np.array(data)
    
    data_w_timesteps = np.zeros((data.shape[0]-lookback, lookback, data.shape[1]), dtype=np.float)
    #print("data_w_timesteps shape: ", data_w_timesteps.shape)
    
    for i in range(data_w_timesteps.shape[0]):
        for i2 in range(10):
            data_w_timesteps[i][i2] = data[i + i2]
    
    return data_w_timesteps
    
# Data with time steps
x_train_tm = create_dataset_with_lookback(x_train, lookback)
x_test_tm = create_dataset_with_lookback(x_test, lookback)

In [ ]:
train_date = train_date.tail(y_train.shape[0]-lookback-1)
test_date  = test_date.tail(y_test.shape[0]-lookback)

x_train    = x_train[1:]
y_train_tm = y_train.tail(y_train.shape[0]-lookback-1)
y_test_tm  = y_test.tail(y_test.shape[0]-lookback)

x_train_tm = create_dataset_with_lookback(x_train, lookback)
x_test_tm = create_dataset_with_lookback(x_test, lookback)

In [ ]:
print("x_train shape: ", x_train_tm.shape)
print("y_train shape: ", y_train_tm.shape)
print("x_test shape: ",  x_test_tm.shape)
print("y_test shape: ",  y_test_tm.shape)

In [ ]:
y_train_tm

In [ ]:
### LSTM model ###

model = Sequential()

model.add(LSTM(units = 32, return_sequences = True, input_shape=(lookback, x_train_tm.shape[2])))
#model.add(Dropout(0.2))



model.add(Dense(units = 1, activation='sigmoid'))

model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics=['accuracy'])

#model.build()
history = model.fit(x_train_tm, 
                    y_train_tm,
                    epochs           = 100, 
                    batch_size       = 16,
                    shuffle          = True,
                    validation_split = 0.2,
                    verbose          = 1)

In [ ]:
print(history.history.keys())

plt.plot(history.history['loss'], label="loss")
plt.plot(history.history['accuracy'], label="accuracy")
plt.plot(history.history['val_loss'], label="val_loss")
plt.plot(history.history['val_accuracy'],label="val_accuracy")
plt.legend()
plt.show()

In [ ]:
### Save train ###

y_train_preds_intervals = model.predict_classes(x_train_tm)
y_train_preds           = []
for pred_interval in y_train_preds_intervals:
    p0 = 0
    p1 = 0
    for pred in pred_interval:
        if pred == 0:
            p0 = p0 + 1
        elif pred == 1:
            p1 = p1 + 1
        else:
            print("Error!!")
            break
    if p1 >= p0:
        y_train_preds.append(1)
    else:
        y_train_preds.append(0)

train_preds = pd.DataFrame({ticket : y_train_preds}, index = train_date)
print(train_preds.value_counts())
train_preds.to_csv("train_signal_" + ticket + ".csv", index=True)


### Save test ### 

y_test_preds_intervals = model.predict_classes(x_test_tm)
y_test_preds           = []
for pred_interval in y_test_preds_intervals:
    p0 = 0
    p1 = 0
    for pred in pred_interval:
        if pred == 0:
            p0 = p0 + 1
        elif pred == 1:
            p1 = p1 + 1
        else:
            print("Error!!")
            break
    if p1 >= p0:
        y_test_preds.append(1)
    else:
        y_test_preds.append(0)

test_preds = pd.DataFrame({ticket : y_test_preds}, index = test_date)
print(test_preds.value_counts())
test_preds.to_csv("test_signal_" + ticket + ".csv", index=True)